In [1]:
import re
import pandas as pd
from pypinyin import lazy_pinyin as lpy, Style
from pywubi import wubi as wb

In [2]:
def fin(strokes_path):
    # 如果返回 0, 则也是在unicode中不存在kTotalStrokes字段
    strokes = []
    with open(strokes_path, 'r') as fr:
        for line in fr:
            strokes.append(int(line.strip()))
    return strokes
def get_stroke(c,strokes):
    unicode_ = ord(c)
    if 13312 <= unicode_ <= 64045:
        return strokes[unicode_ - 13312]
    elif 131072 <= unicode_ <= 194998:
        return strokes[unicode_ - 80338]
    else:
        print("c should be a CJK char, or not have stroke in unihan data.")
        # can also return 0

In [3]:
total = pd.read_csv(r'F:\SCU\大创\python导入\总表.csv')
countyAbbr = pd.read_csv(r'F:\SCU\大创\python导入\县缩写.csv')
cityAbbr = pd.read_csv(r'F:\SCU\大创\python导入\市缩写.csv')
dtError = pd.read_csv(r'F:\SCU\大创\python导入\市中区歧义.csv')
strokes = fin(r'F:\SCU\大创\python导入\strokes.txt')
totalDict = {}
countyDict = {}
cityDict = {}
dteDict = {}
for i in range(len(total)):
    totalDict[total.iloc[i,0]]={
        'province':total.iloc[i,1],
        'city':total.iloc[i,2],
        'county':total.iloc[i,3]
                                }
for i in range(len(countyAbbr)):
    countyDict[countyAbbr.iloc[i,1]]=countyAbbr.iloc[i,0]
for i in range(len(cityAbbr)):
    cityDict[cityAbbr.iloc[i,1]]=cityAbbr.iloc[i,0]
for i in range(len(dtError)):
    dteDict[dtError.iloc[i,1]]=dtError.iloc[i,0]
PyIniDict = {
    'b':'1', 'p':'2', 'm':'3', 'f':'4', 'd':'5', 't':'6', 'n':'7', 'l':'7',
    'g':'8', 'k':'9', 'h':'A', 'j':'B', 'q':'C', 'x':'D', 'zh':'E', 'ch':'F',
    'sh':'G', 'r':'H', 'z':'E', 'c':'F', 's':'G', 'y':'I', 'w':'J', '':'I'
}
PyFinalDict = {
    'a':'1', 'o':'2', 'e':'3', 'i':'4', 'u':'5', 'v':'5', 'yu':'5', 'ai':'6',
    'ei':'7', 'ui':'8', 'ao':'9', 'ou':'A', 'iu':'B', 'ie':'C', 'ye':'C',
    've':'D', 'yue':'D', 'ue':'D', 'er':'E', 'an':'F', 'en':'G', 'in':'H',
    'un':'I', 'ven':'J', 'yun':'J', 'ang':'F', 'eng':'I', 'ing':'H',
    'ong':'K', 'yin':'H','you':'A', 'uang':'L', 'iang':'M', 'iong':'N',
    'ua':'O', 'wen':'I', 'yan':'P', 'ian':'P', 'yang':'M', 'yong':'N',
    'uo':'Q', 'wo':'Q', 'ia':'R', 'ya':'R', 'uan':'S', 'wan':'S', 'wa':'O',
    'iao':'T','yao':'T'
}
strokesDict = {
    0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',
    10:'a',11:'b',12:'c',13:'d',14:'e',15:'f',16:'g',17:'h',18:'i',19:'j',
    20:'k',21:'l',22:'m',23:'n',24:'o',25:'p',26:'q',27:'r',28:'s',29:'t',
    30:'u',31:'v',32:'w',33:'x',34:'y',35:'z',36:'A',37:'B',38:'C',39:'D',
    40:'E',41:'F',42:'G',43:'H',44:'I',45:'J',46:'K',47:'L',48:'M',49:'N',
    50:'O',51:'P',52:'Q',53:'R',54:'S',55:'T',56:'U',57:'V',58:'W',59:'X',
    60:'Y',61:'Z'
}

In [4]:
def findAddr(Addr):
    if '市中区' in Addr:
        for i in range(len(dtError)):
            if dtError.iloc[i,1] in Addr:
                return dteDict[dtError.iloc[i,1]]
    else:
        countyTempAddr = []
        for i in range(len(countyAbbr)):
            if countyAbbr.iloc[i,1] in Addr:
                countyTempAddr.append(countyAbbr.iloc[i,1])
        if len(countyTempAddr)>0:
            fore = countyTempAddr[0]
            for j in range(1,len(countyTempAddr)):
                foreTemp = countyTempAddr[j]
                if Addr.index(foreTemp)<Addr.index(fore):
                    fore = foreTemp
            return countyDict[fore]
        for i in range(len(cityAbbr)):
            if cityAbbr.iloc[i,1] in Addr:
                return cityDict[cityAbbr.iloc[i,1]]
        return None
def printAddr(AddrPAC):
    data = [totalDict[AddrPAC]]
    df = pd.DataFrame(data)
    print(df)
def printAddrList(AddrPACList):
    for i in range(len(AddrPACList)):
        if AddrPACList[i] is None:
            print('{}\t{}\t{}\t{}'.format('nan','nan','nan','nan'))
        else:
            d = totalDict[AddrPACList[i]]
            print('{}\t{}\t{}\t{}'.format(AddrPACList[i],d.get('province'),d.get('city'),d.get('county')))

In [5]:
trainData = pd.read_csv(r'F:\SCU\大创\python导入\训练11-27.csv',header=None)
train2PAC = []
for i in range(len(trainData)):
    train2PAC.append(findAddr(trainData.iloc[i,0]))
# print(train2PAC)
printAddrList(train2PAC)
#
f=open(r'F:\SCU\大创\python导入\训练11-27.txt','w')
for i in range(len(train2PAC)):
    f.writelines(str(train2PAC[i])+'\n')
f.close()

511181	四川省	乐山市	峨眉山市
510725	四川省	绵阳市	梓潼县
512000	四川省	资阳市	nan
513431	四川省	凉山彝族自治州	昭觉县
510811	四川省	广元市	昭化区
510811	四川省	广元市	昭化区
nan	nan	nan	nan
511524	四川省	宜宾市	长宁县
511524	四川省	宜宾市	长宁县
511527	四川省	宜宾市	筠连县
nan	nan	nan	nan
nan	nan	nan	nan
511181	四川省	乐山市	峨眉山市
513227	四川省	阿坝藏族羌族自治州	小金县
513434	四川省	凉山彝族自治州	越西县
513434	四川省	凉山彝族自治州	越西县
511621	四川省	广安市	岳池县
nan	nan	nan	nan
nan	nan	nan	nan
nan	nan	nan	nan
nan	nan	nan	nan
511524	四川省	宜宾市	长宁县
511524	四川省	宜宾市	长宁县
511527	四川省	宜宾市	筠连县
511826	四川省	雅安市	芦山县
511823	四川省	雅安市	汉源县
511823	四川省	雅安市	汉源县
511827	四川省	雅安市	宝兴县
511827	四川省	雅安市	宝兴县
511800	四川省	雅安市	nan
511800	四川省	雅安市	nan
511800	四川省	雅安市	nan
511800	四川省	雅安市	nan
511800	四川省	雅安市	nan
511803	四川省	雅安市	名山区
511800	四川省	雅安市	nan
511722	四川省	达州市	宣汉县
510524	四川省	泸州市	叙永县
nan	nan	nan	nan
511822	四川省	雅安市	荥经县
511822	四川省	雅安市	荥经县
511822	四川省	雅安市	荥经县
513329	四川省	甘孜藏族自治州	新龙县
513227	四川省	阿坝藏族羌族自治州	小金县
513227	四川省	阿坝藏族羌族自治州	小金县
513227	四川省	阿坝藏族羌族自治州	小金县
513401	四川省	凉山彝族自治州	西昌市
513223	四川省	阿坝藏族羌族自治州	茂县
nan	nan	nan	nan
nan	nan	nan	nan
nan	nan	nan	nan
nan	nan	nan	n

In [6]:
def MatchCounty(data):
    PATTERN1 = r'([\u4e00-\u9fa5]{2,5}?(?:省|自治区)){0,1}([\u4e00-\u9fa5]{2,7}?(?:自治州|州|市)){0,1}([\u4e00-\u9fa5]{2,7}?(?:区|自治县|县)){0,1}'
    pattern = re.compile(PATTERN1)
    p3 = ''
    m = pattern.search(data)
    if not m:
        print('None')
    if m.lastindex >= 3:
        p3 = m.group(3)
        p3 = re.sub('区|自治县|县','',p3,1)
    return p3

In [7]:
def SimilarityCal(testList,trainList,SimilarityScore):
    l = len(testList)
    SSCsum = 0
    for i in range(l):
        testStr = testList[i]
        if testStr != '':
            testIni = lpy(testStr,style=Style.INITIALS)[0]
            testIniCode = PyIniDict[testIni]
            testFinal = lpy(testStr,style=Style.NORMAL)[0].replace(testIni,'')
            testFinalCode = PyFinalDict[testFinal]
            testTone = lpy(testStr,style=Style.TONE3)[0][-1]
            TestPy = '%s%s%s' % (testIniCode,testFinalCode,testTone)
            TestWb = str(f'{wb(testStr)[0]:$<4}')
            TestBihua = strokesDict[get_stroke(testStr,strokes)]
            TestSSC = TestPy+TestWb+TestBihua
        else:
            TestSSC = '00000000'

        trainStr = trainList[i]
        if trainStr != '':
            trainIni = lpy(trainStr,style=Style.INITIALS)[0]
            trainIniCode = PyIniDict[trainIni]
            trainFinal = lpy(trainStr,style=Style.NORMAL)[0].replace(trainIni,'')
            trainFinalCode = PyFinalDict[trainFinal]
            trainTone = lpy(trainStr,style=Style.TONE3)[0][-1]
            TrainPy = '%s%s%s' % (trainIniCode,trainFinalCode,trainTone)
            TrainWb = str(f'{wb(trainStr)[0]:$<4}')
            TrainBihua = strokesDict[get_stroke(trainStr,strokes)]
            TrainSSC = TrainPy+TrainWb+TrainBihua
        else:
            TrainSSC = '00000000'

        # print(TestSSC+'\t'+TrainSSC)

        def CComp(char1,char2):
            if char1 == char2: return 1
            else: return 0

        te = list(TestSSC)
        tr = list(TrainSSC)
        SSCscore = 0
        for j in range(len(te)):
            if j in [3,4,5,6]:
                SSCscore += 0.15*CComp(te[j],tr[j])
            else:
                SSCscore += 0.1*CComp(te[j],tr[j])
            # SSCscore += 0.125*CComp(te[j],tr[j])
        SSCsum += SSCscore
    SSCsum /= l
    # SimilarityScore[''.join(trainList)] = SSCsum
    SimilarityScore.append([''.join(trainList),round(SSCsum,1)])

In [8]:
def countyAddrComp(p3):
    # countyPro = {}
    leastRatio = 0.4
    SimilarityScore = []
    TestLen = len(p3)
    for i in range(TestLen):
        for j in range(len(countyAbbr)):
            if p3[i] in countyAbbr.iloc[j,1]:
                trainAbbr = countyAbbr.iloc[j,1]
                LocaTrain = countyAbbr.iloc[j,1].find(p3[i])
                LocaTest = i
                # print(LocaTrain,LocaTest)
                TrainList = list(trainAbbr)
                TestList = list(p3)
                # print(TrainList,TestList)
                diff = LocaTrain - LocaTest
                while diff > 0:
                    TestList.insert(0,"")
                    diff -= 1
                while diff < 0:
                    TrainList.insert(0,"")
                    diff += 1
                lenDiff = len(TrainList)-len(TestList)
                while lenDiff > 0:
                    TestList.append("")
                    lenDiff -= 1
                while lenDiff < 0:
                    TrainList.append("")
                    lenDiff += 1
                SimilarityCal(TestList,TrainList,SimilarityScore)
    SimilarityScore.sort(key=(lambda x: x[1]),reverse=True)
    # print(SimilarityScore)
    if SimilarityScore[0] is not None and SimilarityScore[0][1] > leastRatio:
        return SimilarityScore[0]
    else:
        return None

In [9]:
# data=str(input("请输入文本:"))
temp = '四川省成都市双流区'
p3 = MatchCounty(temp)
result = countyAddrComp(p3)
print(result)

['双流', 1.0]


In [10]:
import jieba
from jieba import analyse
jieba.load_userdict(r'F:\SCU\大创\python导入\city_dict.txt')
jieba.load_userdict(r'F:\SCU\大创\python导入\county_dict.txt')
def MatchCounty_new(data):
    word_bag = []
    # locations = jieba.cut(data,cut_all=False,HMM=False)
    textrank = analyse.textrank
    locations = textrank(data)
    for l in locations:
        word_bag.append(l)
    print(word_bag)

Building prefix dict from the default dictionary ...


KeyboardInterrupt: 

In [ ]:
temp1 = '四川重庆市南岸区黄桷垭镇中药研究所'
location = MatchCounty_new(temp1)